In [ ]:
!pip install diffusers transformers accelerate scipy safetensors

In [ ]:
!pip install flask
!pip install flask_cors
!pip install pyngrok
!pip install pillow

In [ ]:
# stablediffusion运行
import torch
from diffusers import StableDiffusionPipeline
from diffusers import DDIMScheduler,DPMSolverMultistepScheduler,EulerDiscreteScheduler,UniPCMultistepScheduler
model_id = "stabilityai/stable-diffusion-2-1"

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

In [ ]:
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

In [ ]:
# generator = torch.Generator(device="cuda").manual_seed(42)  # 固定随机种子
# image = pipe(
#     prompt="A girl wearing Hanfu, dancing with sword, bamboo forest, moonlight, traditional Chinese aesthetics",
#     negative_prompt="模糊、低质量",  # 排除不良特征
#     num_inference_steps=50,         # 推理步数
#     guidance_scale=7.5,              # 提示词相关性控制
#     generator=generator
# ).images[0]

In [ ]:
# image.save("image5.png")

In [ ]:
# !nvidia-smi

In [ ]:
from flask import Flask, request, jsonify,Response
from flask_cors import CORS
from PIL import Image
import io
import requests,json
import base64
import re

app = Flask(__name__)
CORS(app)  # 允许所有路由的 CORS

@app.route('/hello',methods=['GET'])
def hello():
    return "helloworld!"
    
@app.route('/image', methods=['POST'])
def get_image():
    # 获取 POST 请求中的数据
    data = request.json  #  JSON 格式的数据
    if not data:
        return jsonify({"error": "No data provided"}), 400

    # 处理数据
    prompt = data.get("prompt")
    # print(message)
    # ds_ans,prompt = message_process(message)
    try:
        # 读取图片文件
        # img = Image.open('plate.png')
        generator = torch.Generator(device="cuda").manual_seed(42)  # 固定随机种子
        img = pipe(
            prompt=prompt,
            negative_prompt="模糊、低质量",
            num_inference_steps=20,         # 推理步数
            guidance_scale=7,              # 提示词相关性控制
            generator=generator
        ).images[0]
        # print(type(img))
        # 压缩大小
        img = img.resize((400,400), Image.LANCZOS)
        # 将图片转换为字节流
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='PNG',optimize=True)
        img_byte_arr = img_byte_arr.getvalue()

        # 将字节流转换为 Base64 字符串
        img_base64 = base64.b64encode(img_byte_arr).decode('utf-8')
        # 发送回去
        response = {
          "status": "success",
          "message": "Data received successfully",
          "respond": {
              "img_base64":img_base64
        }
        }
        return jsonify(response), 200

    except Exception as e:
        return jsonify({'error': str(e)}), 500

In [17]:
# 启动 Flask 应用内网穿透
import threading
from pyngrok import ngrok
auth_token = '2tZsI8i2d5znRsyhd73E8mw2PQd_7dYVsgL81ZQdoThvjbc82'
ngrok.set_auth_token(auth_token)
port = 5005
def run_app():
    app.run(port=port)

threading.Thread(target=run_app).start()

# 使用 ngrok 暴露端口
public_url = ngrok.connect(port)
print(f"Flask 应用的公网地址：{public_url}")

 * Serving Flask app '__main__'
 * Debug mode: off
Flask 应用的公网地址：NgrokTunnel: "https://4a33-35-227-150-67.ngrok-free.app" -> "http://localhost:5005"


In [16]:
# 关闭 ngrok 隧道
# ngrok.kill()
# print("隧道已关闭")

In [13]:
!nvidia-smi

Sun May 11 07:02:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P0             30W /   70W |    2627MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [18]:
import time
import datetime
import IPython.display as display

def keep_alive():
    """
    防止Kaggle笔记本休眠的函数
    通过定期执行操作保持笔记本活跃
    """
    try:
        while True:
            # 打印当前时间以显示脚本仍在运行
            now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"笔记本保持运行中... 当前时间: {now}", end="\r")
            
            # 清除输出以保持整洁
            display.clear_output(wait=True)
            
            # 等待一定时间（例如300秒 = 5分钟）
            time.sleep(300)
    except KeyboardInterrupt:
        print("保持活跃的脚本已停止")

# 在后台线程中运行此函数
keep_alive_thread = threading.Thread(target=keep_alive)
keep_alive_thread.daemon = True  # 使线程成为守护线程，这样当主程序退出时它也会退出
keep_alive_thread.start()

print("防休眠程序已在后台启动。您可以继续运行其他代码。")